#### For each good fly, I want to run a few basic analyses:
    - Plot PC 2D Hist maps (first ~50-100 PCs)
    - Foward and Rotational R2 predictions for a distribution of num_PCs
    - Some visualization of which 2D maps are being used by each model
    - Spatial PC distribution for forward and rotation
    
#### Let's order the steps.
    Q1: What is best num_pcs to use across all flies?
        - Load a fly's PCs
        - Predict with different #PCs
        - Save coef figure
        - Append R2s
        - Finally, compare R2 distribution across flies
    Q2: For that num_pcs, compare the 2Dhist plots across flies with high/different weights from forward and turning.
        - 
        
#### Final figures:
    - Show predicted behavior on completely held out data
        - ie, predict with standard CV or whatever, but only give the model 90% of the data!

In [1]:
import numpy as np
from time import time
import os
import sys
import scipy
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from scipy.interpolate import interp1d
import pandas as pd
import psutil
import scipy as sp
import scipy.ndimage
from tqdm import tqdm
sys.path.insert(0, '/home/users/brezovec/.local/lib/python3.6/site-packages/lib/python/')
import ants
import bigbadbrain as bbb
from scipy.linalg import toeplitz
import scipy.linalg as sl
from scipy.signal import convolve2d
from scipy.signal import convolve
import sklearn
from sklearn.linear_model import LassoLarsIC
from sklearn.linear_model import MultiTaskLassoCV
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFE
from sklearn import linear_model
from sklearn.decomposition import FastICA

from skimage.filters import threshold_triangle
sys.path.insert(0, '/home/users/brezovec/.local/lib/python3.6/site-packages')
import cv2

import statsmodels.api as sm

%matplotlib inline
plt.rcParams.update({'font.size': 20})

In [3]:
def interp_fictrac(fictrac, behavior, fps, resolution, expt_len, timestamps, interp_to):
    camera_rate = 1/fps * 1000 # camera frame rate in ms
    sphere_radius = 4.5e-3
    filter_window = 51
    
    x_original = np.arange(0,expt_len,camera_rate)
    
    if behavior == 'all':
        dx = np.asarray(fictrac['dRotLabX'])
        dy = np.asarray(fictrac['dRotLabY'])
        dz = np.asarray(fictrac['dRotLabZ'])
        dx = scipy.signal.savgol_filter(dx,filter_window,3) * sphere_radius * 50 * 100 * 10
        dy = scipy.signal.savgol_filter(dy,filter_window,3) * sphere_radius * 50 * 100 * 10
        dz = scipy.signal.savgol_filter(dz,filter_window,3) * 180 / np.pi * 50
        fictrac_smoothed = np.sqrt(dx*dx + dy*dy + dz*dz)
    elif behavior == 'Y':
        dy = np.asarray(fictrac['dRotLabY'])
        fictrac_smoothed = scipy.signal.savgol_filter(dy,filter_window,3) * sphere_radius * 50 * 100 * 10
    elif behavior == 'Z':
        dz = np.asarray(fictrac['dRotLabZ'])
        fictrac_smoothed = scipy.signal.savgol_filter(dz,filter_window,3) * 180 / np.pi * 50
    else:
        print('invalid behavior')
    
    #fictrac_smoothed = np.abs(fictrac_smoothed)
    fictrac_interp_temp = interp1d(x_original, fictrac_smoothed, bounds_error = False)
    xnew = np.arange(0,expt_len,resolution) #0 to last time at subsample res
    
    if interp_to is 'timestamps':
        fictrac_interp = fictrac_interp_temp(timestamps[:,25])
    elif interp_to is 'xnew':
        fictrac_interp = fictrac_interp_temp(xnew)
    else:
        print('Invalid interp_to ({})'.format(interp_to))

    # Replace Nans with zeros (for later code)
    np.nan_to_num(fictrac_interp, copy=False);
    
    return fictrac_interp

In [4]:
def get_r2(true, prediction):
    u = np.sum((prediction-true)**2)
    v = np.sum((true-np.mean(true))**2)
    return 1 - u / v

In [5]:
root_directory = '/oak/stanford/groups/trc/data/Brezovec/2P_Imaging/20190101_walking_dataset'

In [6]:
flies = ['fly_1',
         'fly_3',
         'fly_5',
         'fly_7',
         'fly_19',
         'fly_21',
         'fly_48',
         'fly_51',
         'fly_54',
         'fly_68']

In [7]:
models = []

for fly in flies:
    directory = os.path.join(root_directory, fly, 'func_0')

    timestamps = bbb.load_timestamps(os.path.join(directory, 'imaging'))
    fictrac = bbb.load_fictrac(os.path.join(directory, 'fictrac'))
    pca_loadings = np.load(os.path.join(directory, 'pca', 'loadings_(temporal).npy'))

    ### Interp fictrac ###

    resolution = 50 #desired resolution in ms
    expt_len = 1000*30*60
    fps = 50 #of fictrac camera
    xnew = np.arange(0,expt_len,resolution)

    fictracs = {}
    for behavior in ['all', 'Y', 'Z']:
        fictracs[behavior] = interp_fictrac(fictrac, behavior, fps, resolution, expt_len, timestamps, 'timestamps')

    pca_loadings_std = np.std(pca_loadings,axis=0)
    pca_loadings= np.divide(pca_loadings,pca_loadings_std)

    for behavior in ['Y', 'Z']:
        fictracs_std = np.std(fictracs[behavior])
        fictracs[behavior] = np.divide(fictracs[behavior],fictracs_std)

    Y_glm = np.vstack((fictracs['Y'], fictracs['Z'])).T

    for num_pcs in [10,25,50,75,100,200,300,500,1000,2000]:
        t0 = time()
        models.append({'num_pcs': num_pcs,
                       'model': MultiTaskLassoCV(),
                       'fly': fly})
        X_glm = pca_loadings[:,:num_pcs]

        # Fit model
        models[-1]['model'].fit(X_glm, Y_glm)

        # Get scores
        prediction = models[-1]['model'].predict(X_glm)
        models[-1]['score_Y'] = get_r2(fictracs['Y'], prediction[:,0])
        models[-1]['score_Z'] = get_r2(fictracs['Z'], prediction[:,1])
        models[-1]['score'] = models[-1]['model'].score(X_glm, Y_glm)

        print('Num PCs: {} | Dur: {:0.0f}s'.format(num_pcs, time()-t0))


~~ load_timestamps ~~
Trying to load timestamp data from hdf5 file.
Success.
load_timestamps done. Duration: 1.36 sec

~~ load_fictrac ~~
load_fictrac done. Duration: 3.36 sec
Num PCs: 10 | Dur: 0s
Num PCs: 25 | Dur: 3s
Num PCs: 50 | Dur: 4s
Num PCs: 75 | Dur: 7s
Num PCs: 100 | Dur: 9s
Num PCs: 200 | Dur: 19s
Num PCs: 300 | Dur: 33s
Num PCs: 500 | Dur: 66s
Num PCs: 1000 | Dur: 180s
Num PCs: 2000 | Dur: 737s

~~ load_timestamps ~~
Trying to load timestamp data from hdf5 file.
Success.
load_timestamps done. Duration: 359.72 ms

~~ load_fictrac ~~
load_fictrac done. Duration: 3.08 sec
Num PCs: 10 | Dur: 0s
Num PCs: 25 | Dur: 1s
Num PCs: 50 | Dur: 2s
Num PCs: 75 | Dur: 3s
Num PCs: 100 | Dur: 4s
Num PCs: 200 | Dur: 8s
Num PCs: 300 | Dur: 13s
Num PCs: 500 | Dur: 23s
Num PCs: 1000 | Dur: 65s
Num PCs: 2000 | Dur: 376s

~~ load_timestamps ~~
Trying to load timestamp data from hdf5 file.
Success.
load_timestamps done. Duration: 1.18 sec

~~ load_fictrac ~~
load_fictrac done. Duration: 3.05 sec


In [8]:
len(models)

100

In [14]:
dir(models[0]['model'])

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_decision_function',
 '_estimator_type',
 '_get_param_names',
 '_preprocess_data',
 '_set_intercept',
 'alpha_',
 'alphas',
 'alphas_',
 'coef_',
 'copy_X',
 'cv',
 'dual_gap_',
 'eps',
 'fit',
 'fit_intercept',
 'get_params',
 'intercept_',
 'max_iter',
 'mse_path_',
 'n_alphas',
 'n_iter_',
 'n_jobs',
 'normalize',
 'path',
 'positive',
 'precompute',
 'predict',
 'random_state',
 'score',
 'selection',
 'set_params',
 'tol',
 'verbose']

In [34]:
models_for_saving[0]

{'coef_Y': [0.006240564478381373,
  -0.03696323110700434,
  0.14803076809136456,
  -0.15173859802020975,
  -0.14873608026794394,
  0.13252173140125548,
  -0.29176597557177414,
  0.025970095376289914,
  -0.036007684667967235,
  0.304120778465223],
 'coef_Z': [-0.007909465988064725,
  -0.01481168304495498,
  0.0023804978116918236,
  -0.08072709164152847,
  -0.018146639385701536,
  -0.02159561494579837,
  -0.027564312286649854,
  -0.026412477101620854,
  -0.002115063041530677,
  0.08179584845213565],
 'fly': 'fly_1',
 'num_pcs': 10,
 'score': 0.15713249817922123,
 'score_Y': 0.29609411843721123,
 'score_Z': 0.018170877921231066}

In [33]:
models_for_saving = []
for model in models:
    new_model = {}
    new_model['fly'] = model['fly']
    new_model['num_pcs'] = model['num_pcs']
    new_model['score'] = model['score']
    new_model['score_Y'] = model['score_Y']
    new_model['score_Z'] = model['score_Z']
    new_model['coef_Y'] = model['model'].coef_[0,:].tolist()
    new_model['coef_Z'] = model['model'].coef_[1,:].tolist()
    models_for_saving.append(new_model)

In [30]:
save_file = os.path.join(root_directory, '20191106_analysis', '20191106_models.json')


In [28]:
import json

In [35]:
with open(save_file, 'w') as outfile:
    json.dump(models_for_saving, outfile)